In [7]:
import time
import json
import time
from riotwatcher import LolWatcher, ApiError, RiotWatcher

In [9]:
# グローバル変数を初期化
request_count = 0  # 現在のリクエスト数
start_time = time.time()  # 開始時間（秒）

def rate_limit_request(request_func, *args, **kwargs):

    global request_count, start_time

    elapsed_time = time.time() - start_time  # 経過時間を計算

    # 1秒あたり20リクエスト制限
    if request_count >= 20 and elapsed_time < 1:
        time.sleep(1 - elapsed_time)  # 必要に応じて1秒まで待機
        start_time = time.time()  # 開始時間をリセット
        request_count = 0  # カウントをリセット

    # 2分あたり100リクエスト制限
    if request_count % 100 == 0 and request_count != 0 and elapsed_time < 120:
        time.sleep(120 - elapsed_time)  # 必要に応じて2分まで待機
        start_time = time.time()  # 開始時間をリセット

    request_count += 1  # リクエストカウントを増加
    return request_func(*args, **kwargs)  # リクエストを実行


In [10]:
api_key = "RGAPI-fe0e79a5-8114-4e91-a0d7-ce6c6e57f5b6"
region = "asia"

riotwatcher = RiotWatcher(api_key)
lolwatcher = LolWatcher(api_key)

In [11]:
# プレイヤー情報（名前とタグライン）
players = [
    {"name": "민철이여친구함", "tagline": "0415"},
    {"name": "M G", "tagline": "2821"},
    {"name": "DK ShowMaker", "tagline": "KR1"},
    {"name": "Yondaime", "tagline": "Luo"},
    {"name": "FATE", "tagline": "KR2"},
]

In [ ]:
# 1. PUUIDを取得
puuids = {}  # プレイヤー名をキーにPUUIDを保存
for player in players:
    pu = riotwatcher.account.by_riot_id(region=region, game_name=player['name'], tag_line=player['tagline'])
    puuids[player['name']] = pu['puuid'] 
    print(f" {player['name']}: {pu['puuid']}")


In [ ]:
# 2. 直近500試合のマッチIDを取得
match_ids = {player['name']: [] for player in players}  # プレイヤー名をキーに試合IDリストを保存

for player in players:
    puuid = puuids.get(player['name'])
    for start in range(0, 500, 100):
        # PUUIDを使って試合リストを取得
        matches = lolwatcher.match.matchlist_by_puuid(region=region, puuid=puuid, start=start, count=100)
        match_ids[player['name']].extend(matches)

# 結果を表示
total_matches = sum(len(matches) for matches in match_ids.values())  # 合計試合数を計算

print(f"合計取得された試合ID数: {total_matches}")
for name, matches in match_ids.items():
    print(f"Player: {name}, Match IDの数: {len(matches)}")
    print(f"Match IDs: {matches}")

In [ ]:
# 出力先のJSONファイル
output_file = 'match_data.json'

# JSONファイルを初期化（空のリストを作成）
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('[')  # JSON配列の開始

# 全ての試合IDをリストとして取得
all_match_ids = [match_id for match_list in match_ids.values() for match_id in match_list]

# 全試合データを取得して保存
for index, match_id in enumerate(all_match_ids):
    # 試合データを取得
    match_data = lolwatcher.match.by_id(region=region, match_id=match_id)

    # 試合データを保存
    with open(output_file, 'a', encoding='utf-8') as file:
        if index > 0:  # 最初の試合データ以外はカンマを追加
            file.write(',\n')
        json.dump(match_data, file, ensure_ascii=False, indent=4)

    print(f"Match ID {match_id} 保存完了 ({index + 1}/{len(all_match_ids)})")

# JSON配列の終了部分を追加
with open(output_file, 'a', encoding='utf-8') as file:
    file.write('\n]')  # JSON配列の終了

print("全ての試合データの保存完了")


In [ ]:
# 出力先のJSONファイル
output_file = 'timeline_data.json'

# JSONファイルを初期化（空のリストを作成）
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('[')  # JSON配列の開始

# 全ての試合IDをリストとして取得
all_match_ids = [match_id for match_list in match_ids.values() for match_id in match_list]

# 全試合データを取得して保存
for index, match_id in enumerate(all_match_ids):
    # 試合データを取得
    timeline_data = lolwatcher.match.timeline_by_match(region=region, match_id=match_id)

    # 試合データを保存
    with open(output_file, 'a', encoding='utf-8') as file:
        if index > 0:  # 最初の試合データ以外はカンマを追加
            file.write(',\n')
        json.dump(timeline_data, file, ensure_ascii=False, indent=4)

    print(f"Match ID {match_id} 保存完了 ({index + 1}/{len(all_match_ids)})")

# JSON配列の終了部分を追加
with open(output_file, 'a', encoding='utf-8') as file:
    file.write('\n]')  # JSON配列の終了

print("全ての試合データの保存完了")